In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from skimage import io
from skimage import morphology
from tifffile import imsave
from tqdm import tqdm
from time import time
from skimage.measure import block_reduce
from math import ceil
from scipy import ndimage as ndi
import glob
import os
import pandas as pd
from scipy.stats import ttest_ind, mannwhitneyu
from skan import Skeleton, summarize
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

def image_show(image, nrows=1, ncols=1, cmap='gray',size = 8):
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(size, size))
    ax.imshow(image, cmap='gray')
    ax.axis('off')
    plt.show();
    return fig, ax

def vimg_show(volume):
    image_show(volume[volume.shape[0]//2])
    
def create_sphere(radius):  #3d sphere

    Z, Y, X = np.ogrid[:(2*radius)+1, :(2*radius)+1, :(2*radius)+1]
    dist_from_center = np.sqrt((X - radius)**2 + (Y-radius)**2 + (Z - radius)**2)

    mask = (dist_from_center <= radius)
    return mask.astype(np.bool)

In [3]:
# Identify cancer biospies:
root = 'E:\\Weisi_data'
folders = ['send_Can_data_111620', 'send_Can_data_121120','send_Can_data_011521','send_Can_data_012921','send_Can_data_021321','send_Can_data_022821','send_Can_data_030421']
ca_biopsies = [['018a','018c','018e','021b','021c','029b','029c','042d','042e','025b','028b'],
               ['040b','040f','034b','032a','002b','002d','002e','002f','003a','003b','003f','004a','004b','004d','004e','011c','012b','012d','009a','007a','007b','007d'], #'007a','007b','007d'
               ['001a','001b','001c','001d','008f','014d','015b','015c','015d','006c','026a','026b','031a','031b','031c','031d'],
               ['044a','088a','088b','088c','088f','062b','062d','062e','062f','047b','057c','013b','013c','023b','023d'],
               ['065a','065c','068b','068d','068f','005b','005d','005f','038c','038f','094d','094e','094f','106a','112b','112d','112e','116d','116e','066c','066e'], #'066c','066e'
               ['123a','123b','123f','126d','143c','100c','100d','100e','100f','125c','125d','093b','093c','093d','142a','142b','142e','142f','092d','092f','144a','144c','144d','146a','146c','146e','148a','148b','148c','148e'],
               ['143e','144b','144f']]

In [ ]:
# Check if all bioipsy exists and count total number of biopsies and cases

exist_bipsy_list = []
exist_case_list = []

for fi, folder in enumerate(folders): #check
    folder_dir = os.path.join(root, folder)
    
    for bi, biopsy in enumerate(ca_biopsies[fi]): #check
        print('----', 'dod_s'+biopsy)
        
        biopsy_dir = os.path.join(folder_dir, 'dod_s'+biopsy+'.npz')
        
        if not os.path.exists(biopsy_dir):
            print("!!! doesn't exist directory:", biopsy_dir)
        else:
            if biopsy in exist_bipsy_list:
                print("??? duplicated biopsy:", biopsy)
            else:
                exist_bipsy_list.append(biopsy)
            if not biopsy[:3] in exist_case_list:
                exist_case_list.append(biopsy[:3])
                
print('total number of biopsies:', len(exist_bipsy_list))   
print('total number of cases:', len(exist_case_list))

In [ ]:
# Import recurrence code
excel_dir = os.path.join(root, 'Liu_Lab_Canary_data.xlsx')
BCR_data = pd.read_excel(excel_dir, sheet_name='ParticipantData', header=0, usecols=[0,2], names=['case#', 'BCR'])
# print(BCR_data)
BCR_dict = BCR_data.set_index('case#')['BCR'].to_dict() #create a dictionary of BCR
print(BCR_dict)

print('BCR_dict[200]=',BCR_dict[200])

In [ ]:
# Import cancer region
region_dir = os.path.join(root,'biopsy_cancer_region.xlsx')
region_data = pd.read_excel(region_dir, sheet_name = 'Sheet1', header=0, usecols=[2,3,4,5,6], names = ['biopsy#','r1_b','r1_t','r2_b','r2_t'])
region_dict = region_data.set_index('biopsy#').T.to_dict('list') #create a dictionary of cancer regions
print(region_dict)
print('region_dict[011c]=', region_dict['011c'])
region = region_dict['011c']
print(len(region))
print(np.isnan(region[3]))

In [ ]:
####################
####################calculate 3d branching features#####################

# specify parameters
feature_of_interest = 'lumens'
erosion_round = 1
dilation_round = 0
kern_rad = 2
ds_time = 4

#################### start processing

c_or_b_list = ['cancer'] #benign/cancer 'cancer','benign'

for c_or_b in c_or_b_list:

    save_file_name = c_or_b+'_'+feature_of_interest+'_3d_open'+str(erosion_round)+str(dilation_round)+'rad'+str(kern_rad)+'_ds'+str(ds_time)+'.txt'


    if os.path.exists(os.path.join(root,save_file_name)):
        os.remove(os.path.join(root,save_file_name))

    for fi, folder in tqdm(enumerate(folders[3:4])): #check
        folder_dir = os.path.join(root, folder)

        for bi, biopsy in enumerate(ca_biopsies[fi+3][-1:]): #check
            print('----', 'dod_s'+biopsy)

            biopsy_dir = os.path.join(folder_dir, 'dod_s'+biopsy+'.npz')
            case_n = int(biopsy[:3]) #int delete leading zeros


            #########################################gland features

            #######---------import gland structure file

            with np.load(biopsy_dir) as data:
                gland = data[feature_of_interest].astype(np.bool)
    #         print(gland.dtype)
    #         print(gland.shape)

            #######---------extract cancer region (eliminating everythin outside the cancer region)

            cancer_region = region_dict[biopsy] # a list of 4 elements
            cancer_region[0] = min(cancer_region[0], gland.shape[1]-1)
            if not np.isnan(cancer_region[2]): #if there is a second region
                cancer_region[2] = min(cancer_region[2], gland.shape[1]-1)

            cancer_gland = np.copy(gland)
            if np.isnan(cancer_region[2]): #only 1 region
                cancer_gland[:,:int(cancer_region[1]),:]=0 #set 0-r1_t to 0
                cancer_gland[:,int(cancer_region[0]):,:]=0 #set r1_b to bottom to 0
            else: #2 regions
                #make sure region 2 < region 1
                cancer_gland[:,:int(cancer_region[3]),:]=0 #set 0-r2_t to 0
                cancer_gland[:,int(cancer_region[2]):int(cancer_region[1]),:]=0 #set r2_b to r1_t to 0
                cancer_gland[:,int(cancer_region[0]):,:]=0 #set r1_b to bottom to 0


            #######---------extract benign region
            if c_or_b == 'cancer':
                c_or_b_gland =  cancer_gland
            else:
                benign_gland = np.bitwise_and(gland, np.invert(cancer_gland))
                c_or_b_gland =  benign_gland
    

            #######---------downsample nX

            gland_ds_ori = block_reduce(c_or_b_gland, block_size=(ds_time, ds_time, ds_time), func=np.mean).astype(np.bool)

            #######---------erosion and dilation before skeletonize

            gland_ds = ndi.binary_erosion(gland_ds_ori, structure = create_sphere(kern_rad), iterations = erosion_round)
            #the erosion operation removes objects that are smaller than structuring element, 
            #and the dilation operation (approximately) restores the size and shape of the remaining objects.

            print(gland_ds.dtype)
            print(gland_ds.shape)
            vimg_show(gland_ds)

            #######---------skeletonize

            if np.count_nonzero(gland_ds)==0: #no structure
                print('no structure')
                f= open(os.path.join(root,save_file_name),"a") #append mode
                f.write("s"+biopsy+","+str(BCR_dict[case_n])+","+str(0)+","+str(0)+','+str(1)+','+str(1)+','+str(0)+','+str(0)+"\n")
                f.close()
                
            else:
                try:
                    # Skeletonize
                    skel_1 = morphology.skeletonize_3d(gland_ds)
                    fill_hole = ndi.morphology.binary_fill_holes(skel_1) #fill 3D empty regions
                    skel_2 = morphology.skeletonize_3d(fill_hole)
    
                #######---------branching feature extraction

                    branches_all = summarize(Skeleton(skel_2.astype(np.bool)))
            
                except:
                    print('current biopsy is not skeletonizable...')
                    f= open(os.path.join(root,save_file_name),"a") #append mode
                    f.write("s"+biopsy+","+str(BCR_dict[case_n])+","+str(0)+","+str(0)+','+str(1)+','+str(1)+','+str(0)+','+str(0)+"\n")
                    f.close()
                    
                    continue # if skeletonization fails, keep features 0, restart loop with next level


                ###-----features for all branches

                all_branch_dist_mean = branches_all["branch-distance"].mean()
                if np.isnan(all_branch_dist_mean):
                    all_branch_dist_mean = 0
                print('all branch_dist_mean', all_branch_dist_mean)

                all_branch_dist_std = branches_all["branch-distance"].std()
                if np.isnan(all_branch_dist_std):
                    all_branch_dist_std = 0
                print('all branch_dist_std', all_branch_dist_std)

                branches_all["tortuosity"] = branches_all["branch-distance"]/branches_all["euclidean-distance"]
                branches_all["tortuosity"] = branches_all["tortuosity"].replace({np.inf:np.nan, 0:1})
                
                all_tor_median = branches_all["tortuosity"].median()
                if all_tor_median ==0 or np.isnan(all_tor_median):
                    all_tor_median = 1
                print('all tortuosity_median', all_tor_median)
                
                all_tor_mean = branches_all["tortuosity"].mean()
                if all_tor_mean == 0 or np.isnan(all_tor_mean):
                    all_tor_mean = 1
                print('all tortuosity_mean', all_tor_mean)
                
                all_tor_std = branches_all["tortuosity"].std()
                if np.isnan(all_tor_std):
                    all_tor_std = 0
                print('all tortuosity_std', all_tor_std)

                all_skeleton_count=len(set(branches_all["skeleton-id"])) #count unique values
                all_branch_count = len(branches_all.index)
                if all_skeleton_count>0:
                    all_branch_density_per_skel = all_branch_count/all_skeleton_count
                else:
                    all_branch_density_per_skel = 0
                print('all branch_density_per_skel', all_branch_density_per_skel)


                ###-----save data
                f= open(os.path.join(root,save_file_name),"a") #append mode
                f.write("s"+biopsy+","+str(BCR_dict[case_n])+"," \
                        +str(all_branch_dist_mean)+","+str(all_branch_dist_std)+','+str(all_tor_median)+','+str(all_tor_mean)+','+str(all_tor_std)+','+str(all_branch_density_per_skel)+"\n")
                        
                f.close()